In [ ]:
import pdfplumber
import pandas as pd 
import json 
import os
import re
import csv
import PyPDF2
import numpy as np 

SAMPLE CONVERSION OF PDF TO CSV TABULAR DOCUMENT

In [ ]:
def pdf_table_to_csv(pdf_path, csv_path):
    """
    Extracts tables from a PDF using pdfplumber, converts them to a pandas DataFrame,
    and saves the DataFrame to a CSV file.

    Args:
        pdf_path (str): The path to the PDF file.
        csv_path (str): The path where the CSV file will be saved.

    Returns:
        bool: True if data was successfully extracted and saved, False otherwise.
    """
    all_extracted_data = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables()

                if tables:
                    for table in tables:
                         if table and len(table) > 1:
                            header = [h.replace('\n', ' ').strip() if h else '' for h in table[0]]
                            for row in table[1:]:
                                row_data = {}
                                padded_row = row + [None] * (len(header) - len(row))
                                for i, col_name in enumerate(header):
                                    cell_value = padded_row[i]
                                    row_data[col_name] = cell_value.replace('\n', ' ').strip() if isinstance(cell_value, str) else cell_value
                                all_extracted_data.append(row_data)
                else:
                    print(f"No tables automatically detected on page {page.page_number}. You might need custom text parsing.")


    except FileNotFoundError:
        print(f"Error: File not found at {pdf_path}")
        return False
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

    # --- New part for CSV ---
    if all_extracted_data:
        df = pd.DataFrame(all_extracted_data)
        df.to_csv(csv_path, index=False)
        print(f"Successfully extracted data and saved to {csv_path}")
        return True
    else:
        print("No data extracted from tables.")
        return False

pdf_file_path = 'IRI-07314-Kollam - SSS Hubballi Special Fare Sabarimala Special.pdf'
csv_output_path = 'output_schedule.csv' 

success = pdf_table_to_csv(pdf_file_path, csv_output_path)

if not success:
    print("Process failed.")

TAKING TRAIN DATA FROM PDFs AND TABULATING IT AS DIFFERENT PARAMETERS

In [ ]:
def extract_train_number_from_filename(filename):
    """
    Extracts a cleaned train number (numeric only, no leading zeros) from a PDF filename.
    Finds the first sequence of digits in the filename and removes leading zeros.
    Discards any surrounding alphabets.
    E.g., 'xx1462.pdf' -> '1462', '01462x.pdf' -> '1462', 'IRI-07043-Name.pdf' -> '7043', 'train123.pdf' -> '123', '1462XXX.pdf' -> '1462'
    Returns None if no digit sequence is found in the filename.
    """
    match = re.search(r'(\d+)', filename)
    if match:
        numeric_str = match.group(1)
        try:
            cleaned_train_num = str(int(numeric_str))
            return cleaned_train_num
        except ValueError:
            print(f"Warning: Could not convert '{numeric_str}' to integer from filename {filename}")
            return None
    else:
        return None

def pdf_table_to_csv(pdf_path, csv_path):
    """
    Extracts tables from a PDF using pdfplumber, converts them to a pandas DataFrame,
    renames the first column to 'Zone', and saves the DataFrame to a CSV file.

    Args:
        pdf_path (str): The path to the PDF file.
        csv_path (str): The path where the CSV file will be saved.

    Returns:
        bool: True if data was successfully extracted and saved, False otherwise.
    """
    all_extracted_data = []
    pdf_filename = os.path.basename(pdf_path) 

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                tables = page.extract_tables(table_settings={})

                if tables:
                    for table in tables:
                         if table and len(table) > 1:
                            header = [h.replace('\n', ' ').strip() if h else '' for h in table[0]]
                            for row in table[1:]:
                                row_data = {}
                                padded_row = row + [None] * (len(header) - len(row))
                                for i, col_name in enumerate(header):
                                    cell_value = padded_row[i]
                                    row_data[col_name] = cell_value.replace('\n', ' ').strip() if isinstance(cell_value, str) else cell_value
                                all_extracted_data.append(row_data)


    except FileNotFoundError:
        print(f"Error in pdf_table_to_csv: PDF file not found at {pdf_path}")
        return False
    except Exception as e:
        print(f"An error occurred processing {pdf_filename}: {e}")
        return False

    if all_extracted_data:
        try:
            df = pd.DataFrame(all_extracted_data)
            if not df.empty and not df.columns.empty:
                current_columns = list(df.columns)
                current_columns[0] = 'Zone'
                df.columns = current_columns

            df.to_csv(csv_path, index=False, encoding='utf-8')
            return True
        except Exception as e:
            print(f"Error saving data to CSV {os.path.basename(csv_path)}: {e}")
            return False
    else:
        return False

pdf_folder = "pdfs" 
output_csv_folder = "extracted_tables_csv" 

if not os.path.exists(output_csv_folder):
    os.makedirs(output_csv_folder)
    print(f"Created output folder: {output_csv_folder}")

all_files_in_folder = os.listdir(pdf_folder)

pdf_files = [f for f in all_files_in_folder if f.lower().endswith(".pdf")]

if pdf_files:
    print(f"Found {len(pdf_files)} PDF files in '{pdf_folder}'. Starting table extraction to CSV...")
    pdf_files.sort()

    processed_count = 0
    failed_count = 0

    for i, filename in enumerate(pdf_files):
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"Processing file {i+1}/{len(pdf_files)}: {filename}...")
        train_number = extract_train_number_from_filename(filename)

        if train_number:
            csv_filename = f"{train_number}_table_data.csv"
            csv_output_path = os.path.join(output_csv_folder, csv_filename)
            success = pdf_table_to_csv(pdf_path, csv_output_path)

            if success:
                print(f"  Successfully saved table data to {csv_filename}")
                processed_count += 1
            else:
                print(f"  Table data extraction failed or no data found in {filename}.")
                failed_count += 1

        else:
            print(f"  Could not extract a valid train number from filename: {filename}. Skipping table extraction for this file.")
            failed_count += 1 

    print("\n--- Automation Summary ---")
    print(f"Total PDF files found: {len(pdf_files)}")
    print(f"Files processed successfully (CSV saved): {processed_count}")
    print(f"Files failed or skipped: {failed_count}")
    print(f"Output CSV files saved to: {output_csv_folder}")

else:
    print(f"No PDF files found in the '{pdf_folder}' folder to process.")

SAMPLE DATA PARSING FROM PDF TO CHECK IF ITS WORKING FOR ONE TRAIN 

In [ ]:
def read_train_numbers_from_csv(train_num_csv):
    """Reads train numbers from a CSV file."""
    train_numbers = []
    try:
        with open(train_num_csv, 'r', newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader, None)  
            for row in reader:
                train_numbers.append(row[0])  
        return train_numbers
    except FileNotFoundError:
        print(f"Error: CSV file not found at {train_num_csv}")
        return []
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return []

def find_pdf_by_train_number(train_number, pdf_folder):
    """Finds a PDF file in a folder that contains the train number in its name."""
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            #  Adjust this regex if your filenames have a different pattern
            if re.search(rf"{train_number}", filename, re.IGNORECASE):
                return os.path.join(pdf_folder, filename)
    return None

def extract_train_data(pdf_path, train_num):
    """Extracts train data from a PDF file."""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() or ''

    data = {}
    data['Train Number'] = train_num

    match = re.search(r'(\d+)/([^-]+) - ([^\n]+)', text)
    if match:
        data['Full Train Name'] = match.group(2).strip() + " - " + match.group(3).strip()
        data['Train Type'] = "Mail/Express" 
    else:
        data['Full Train Name'] = None
        data['Train Type'] = None

    match = re.search(r'(\d+h \d+m) - (\d+ km) - (\d+ halts)', text)
    if match:
        duration_str = match.group(1)
        hours, minutes = map(int, duration_str.replace('h ', ' ').replace('m', '').split())
        data['Duration (minutes)'] = hours * 60 + minutes
        data['Distance (km)'] = int(match.group(2).replace(' km', ''))
        data['Number of Halts'] = int(match.group(3).replace(' halts', ''))
    else:
        data['Duration (minutes)'] = None
        data['Distance (km)'] = None
        data['Number of Halts'] = None

    match = re.search(r'Departs ([\w, ]+)', text)
    if match:
        data['Days of Operation'] = match.group(1)
    else:
        data['Days of Operation'] = None

    stations_data = []
    start_extract = False
    for line in text.split('\n'):
        if '"#","Code","Station Name","Arr","Dep","Halt","PF","Day","Km","Spd","Elv","Zone"' in line:
            start_extract = True
            continue
        if start_extract:
            values = line.split('","')
            if len(values) >= 12:  
                try:
                    station = {
                        '#': values[0].replace('"', ''),
                        'Code': values[1].replace('"', ''),
                        'Station Name': values[2].replace('"', ''),
                        'Arr': values[3].replace('"', ''),
                        'Dep': values[4].replace('"', ''),
                        'Halt': values[5].replace('"', ''),
                        'Km': values[8].replace('"', ''),
                        'Spd': values[9].replace('"', ''),
                        'Elv': values[10].replace('"', '')
                    }
                    stations_data.append(station)
                except IndexError:
                    print(f"Incomplete data in line: {line}")
    return data

train_num_csv = "Unique_Trains_in_Passengers - Sheet1.csv"
pdf_folder = "pdfs"

train_numbers = read_train_numbers_from_csv(train_num_csv)

if train_numbers:
    first_train_number = train_numbers[0]  
    pdf_path = find_pdf_by_train_number(first_train_number, pdf_folder)

    if pdf_path:
        extracted_data = extract_train_data(pdf_path, first_train_number)
        print("Extracted Data:\n", extracted_data)  
    else:
        print(f"No PDF found for train number: {first_train_number}")
else:
    print("No train numbers found in the CSV.")

AUTOMATION FOR ALL TRAINS TO GET EXTRACTED DATA FROM PDFS - INCLUDING THE FOLLOWING 
Extracted Data:
 {'Train Number': '1462', 'Full Train Name': 'Kanniyakumari - Mumbai CSMT Special Fare Summer Special', 'Train Type': 'Mail/Express', 'Duration (minutes)': 2205, 'Distance (km)': 1826, 'Number of Halts': 31, 'Days of Operation': 'Thu '}

In [ ]:
def extract_train_number_from_filename(filename):
    """
    Extracts a cleaned train number (numeric only, no leading zeros) from a PDF filename.
    Finds the first sequence of digits in the filename and removes leading zeros.
    Discards any surrounding alphabets.
    E.g., 'xx1462.pdf' -> '1462', '01462x.pdf' -> '1462', 'IRI-07043-Name.pdf' -> '7043', 'train123.pdf' -> '123', '1462XXX.pdf' -> '1462'
    Returns None if no digit sequence is found in the filename.
    """
    match = re.search(r'(\d+)', filename)
    if match:
        numeric_str = match.group(1)
        cleaned_train_num = str(int(numeric_str))
        return cleaned_train_num
    else:
        return None
    
def extract_train_data_from_pdf_content(pdf_path, train_num):
    """
    Extracts high-level train data from the introductory text of a PDF file.
    Attempts to extract Name, Type, Duration, Distance, Halts, Days.
    Initializes placeholders for table-derived data.

    Args:
        pdf_path (str): The full path to the PDF file.
        train_num (str): The cleaned train number extracted from the filename.

    Returns:
        dict: A dictionary containing the extracted and initialized train information.
    """
    data = {'Train Number': train_num,
            'Full Train Name': None,
            'Train Type': None, 
            'Duration (minutes)': None,
            'Distance (km)': None,
            'Number of Halts': None,
            'Days of Operation': None}

    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            if len(reader.pages) > 0:
                 text = reader.pages[0].extract_text() or ''

        match_name = re.search(rf'{re.escape(str(train_num))}/(.+?)(?: Type: |\n)', text)
        if match_name:
             data['Full Train Name'] = match_name.group(1).strip()
        else:
            first_line_match = re.match(r'.+/(.+)', text.split('\n')[0])
            data['Full Train Name'] = first_line_match.group(1).strip() if first_line_match else None

        match_type = re.search(r'Type:\s*(.+?)(?: Zone:|\n|$)', text)
        if match_type:
            data['Train Type'] = match_type.group(1).strip()
        else:
            data['Train Type'] = None

        match_details = re.search(r'(\d+h\s*\d+m)\s*-?\s*(\d+)\s*km\s*-?\s*(\d+)\s*halts', text)
        if match_details:
            duration_str = match_details.group(1)
            try:
                duration_str_cleaned = duration_str.replace('h', ' ').replace('m', '').strip()
                hours, minutes = map(int, duration_str_cleaned.split())
                data['Duration (minutes)'] = hours * 60 + minutes
            except ValueError:
                 data['Duration (minutes)'] = None

            try:
                data['Distance (km)'] = int(match_details.group(2))
            except ValueError:
                 data['Distance (km)'] = None

            try:
                data['Number of Halts'] = int(match_details.group(3))
            except ValueError:
                 data['Number of Halts'] = None
        else:
             data['Duration (minutes)'] = None
             data['Distance (km)'] = None
             data['Number of Halts'] = None


        match_days = re.search(r'Departs ([\w, ]+)', text)
        if match_days:
            data['Days of Operation'] = match_days.group(1).strip()
        else:
             match_days_alt = re.search(r'Zone: \w+ Departs ([\w, ]+)', text)
             if match_days_alt:
                  data['Days of Operation'] = match_days_alt.group(1).strip()
             else:
                data['Days of Operation'] = None

    except FileNotFoundError:
        print(f"Error: PDF file not found at {pdf_path}")
        pass 

    except Exception as e:
        print(f"Error processing PDF {pdf_path}: {e}")
        pass 


    return data

pdf_folder = "pdfs" 
output_csv_file = "all_pdfs_extracted_data.csv" 

all_trains_data = [] 

all_files_in_folder = os.listdir(pdf_folder)

pdf_files = [f for f in all_files_in_folder if f.lower().endswith(".pdf")]

if pdf_files:
    print(f"Found {len(pdf_files)} PDF files in '{pdf_folder}'.")
    pdf_files.sort()

    for i, filename in enumerate(pdf_files):
        pdf_path = os.path.join(pdf_folder, filename)
        print(f"Processing file {i+1}/{len(pdf_files)}: {filename}...")

        train_number = extract_train_number_from_filename(filename)

        if train_number:
            print(f"  Extracted train number from filename: {train_number}")
            extracted_data = extract_train_data_from_pdf_content(pdf_path, train_number)
            all_trains_data.append(extracted_data)
        else:
            print(f"  Could not extract a valid train number from filename: {filename}. Skipping this file.")
            all_trains_data.append({'Train Number': None, 
                                   'Full Train Name': None, 'Train Type': None,
                                   'Duration (minutes)': None, 'Distance (km)': None,
                                   'Number of Halts': None, 'Days of Operation': None,
                                   'Stations': [], 'Origin': None, 'Destination': None,
                                   'Dep Time': None, 'Arr Time': None,
                                   'Total Halt Time (minutes)': None, 'Average Speed (km/h)': None,
                                   'Average Elevation (m)': None, 'Zone of Origin': None,
                                   'Zone of Destination': None})

    if all_trains_data:
        print(f"\nCollected data for {len(all_trains_data)} files. Saving to CSV...")
        df_output = pd.DataFrame(all_trains_data)

        output_columns = [
            'Train Number', 'Full Train Name', 'Train Type', 'Duration (minutes)',
            'Distance (km)', 'Number of Halts', 'Days of Operation', 'Stations',
            'Origin', 'Destination', 'Dep Time', 'Arr Time', 'Total Halt Time (minutes)',
            'Average Speed (km/h)', 'Average Elevation (m)', 'Zone of Origin', 'Zone of Destination'
        ]

        df_output = df_output.reindex(columns=output_columns)

        try:
            df_output.to_csv(output_csv_file, index=False, encoding='utf-8')
            print(f"Successfully saved extracted data to {output_csv_file}")
        except Exception as e:
            print(f"Error saving data to CSV {output_csv_file}: {e}")


    else:
        print("No data was extracted from any PDF file to save.")

else:
    print(f"No PDF files found in the '{pdf_folder}' folder.")

POPULATING DERIVED DATA FROM THE TRAIN SPECIFIC CSVS TO MAIN CSV OF TRAINS

In [ ]:
main_csv_path = "all_pdfs_extracted_data.csv"
extracted_tables_folder = "extracted_tables_csv"

print(f"Attempting to populate data into the main CSV: {main_csv_path}")

try:
    if not os.path.exists(main_csv_path):
        print(f"Error: Main CSV file not found at {main_csv_path}")
    else:
        df_main = pd.read_csv(main_csv_path)

        if 'Stations' in df_main.columns:
            df_main = df_main.drop(columns=['Stations'])
            print("Dropped the 'Stations' column from the main DataFrame.")
        else:
            print("Warning: 'Stations' column not found in the main DataFrame.")

        # --- Check if required columns exist in the main DataFrame ---
        required_main_cols = ['Train Number', 'Origin', 'Destination']
        if not all(col in df_main.columns for col in required_main_cols):
            missing = [col for col in required_main_cols if col not in df_main.columns]
            print(f"Error: Missing required columns in the main CSV: {missing}")
        elif df_main.empty:
             print("Main CSV DataFrame is empty.")
        else:
            print(f"Processing {len(df_main)} trains to populate Origin and Destination...")
            for index, row in df_main.iterrows():
                train_number = row['Train Number']

                if pd.notna(train_number) and train_number != '':
                    train_csv_filename = f"{train_number}_table_data.csv"
                    train_csv_path = os.path.join(extracted_tables_folder, train_csv_filename)

                    if os.path.exists(train_csv_path):
                        try:
                            df_train = pd.read_csv(train_csv_path)

                            if not df_train.empty and 'Station Name' in df_train.columns:
                                origin_station = df_train.iloc[0]['Station Name']

                                destination_station = df_train.iloc[-2]['Station Name']
                                df_main.loc[index, 'Origin'] = origin_station
                                df_main.loc[index, 'Destination'] = destination_station
                                print(f"  Updated Origin and Destination for Train {train_number}") 

                            else:
                                print(f"  Warning: Train-specific CSV {train_csv_filename} is empty or missing 'Station Name' column.")

                        except Exception as e:
                            print(f"  Error reading or processing train-specific CSV {train_csv_filename}: {e}")
                    else:
                        pass 

                else:
                    print(f"  Warning: Skipping row {index} in main CSV due to invalid Train Number: {train_number}")
            print("\nFinished processing. Saving updated main CSV...")
            df_main.to_csv(main_csv_path, index=False, encoding='utf-8')
            print(f"Successfully updated and saved the main CSV file: {main_csv_path}")

except FileNotFoundError:
    print(f"An error occurred because the main CSV file was not found at {main_csv_path}")
except Exception as e:
    print(f"An unexpected error occurred during the population process: {e}")

In [ ]:
main_csv_path = "all_pdfs_extracted_data.csv"
extracted_tables_folder = "extracted_tables_csv"

print(f"Starting population of Dep Time, Arr Time, and Total Halt Time in main CSV: {main_csv_path}")

try:
    df_main = pd.read_csv(main_csv_path)
    if not df_main.empty:
        print(f"Processing {len(df_main)} trains...")
        for index, row in df_main.iterrows():
            train_number = row['Train Number']
            if pd.notna(train_number) and str(train_number).strip() != '':
                train_csv_filename = f"{str(train_number).strip()}_table_data.csv"
                train_csv_path = os.path.join(extracted_tables_folder, train_csv_filename)
                try:
                    df_train = pd.read_csv(train_csv_path)
                    dep_time_val = df_train.iloc[0]['Dep']
                    df_main.loc[index, 'Dep Time'] = str(dep_time_val).strip() if pd.notna(dep_time_val) else None
                    arr_time_val = df_train.iloc[-2]['Arr']
                    df_main.loc[index, 'Arr Time'] = str(arr_time_val).strip() if pd.notna(arr_time_val) else None
                    total_halt_minutes = 0
                    try:
                        for halt_entry in df_train['Halt'].astype(str):
                            if halt_entry.lower() != 'nan' and halt_entry.strip() != '':
                                match = re.match(r'(\d+)m', halt_entry.strip(), re.IGNORECASE)
                                if match:
                                    total_halt_minutes += int(match.group(1))
                    except Exception as e:
                         print(f"  Error calculating total halt time for train {train_number}: {e}")
                         total_halt_minutes = None 


                    df_main.loc[index, 'Total Halt Time (minutes)'] = total_halt_minutes
                    print(f"  Successfully processed times and halt time for train {train_number}") # Optional print


                except FileNotFoundError:
                    print(f"  Warning: Train-specific CSV not found for train {train_number} at {train_csv_path}. Skipping.")
                except Exception as e:
                    print(f"  Error processing train-specific CSV {train_csv_filename} for train {train_number}: {e}. Skipping.")

            print("\nFinished processing. Saving updated main CSV...")
            df_main.to_csv(main_csv_path, index=False, encoding='utf-8')
            print(f"Successfully updated and saved the main CSV file: {main_csv_path}")

    else:
        print("Main CSV DataFrame is empty. Nothing to process.")

except FileNotFoundError:
    print(f"An error occurred because the main CSV file was not found at {main_csv_path}")
except Exception as e:
    print(f"An unexpected error occurred during the population process: {e}")

print("Population script finished.")

In [ ]:
print(f"Starting population of Average Speed and Average Elevation in main CSV: {main_csv_path}")

try:
    df_main = pd.read_csv(main_csv_path)
    required_main_cols = ['Train Number', 'Average Speed (km/h)', 'Average Elevation (m)']
    if not all(col in df_main.columns for col in required_main_cols):
         missing = [col for col in required_main_cols if col not in df_main.columns]
         print(f"Error: Missing required columns in the main CSV for population: {missing}. Please ensure these columns exist.")
    elif df_main.empty:
         print("Main CSV DataFrame is empty. Nothing to process.")
    else:
        print(f"Processing {len(df_main)} trains to populate average speed and elevation...")
        for index, row in df_main.iterrows():
            train_number = row['Train Number']
            if pd.notna(train_number) and str(train_number).strip() != '':
                train_csv_filename = f"{str(train_number).strip()}_table_data.csv"
                train_csv_path = os.path.join(extracted_tables_folder, train_csv_filename)
                try:
                    df_train = pd.read_csv(train_csv_path)
                    df_train['Spd_numeric'] = pd.to_numeric(df_train['Spd'], errors='coerce')
                    average_speed = df_train['Spd_numeric'].mean()
                    df_main.loc[index, 'Average Speed (km/h)'] = round(average_speed, 2) if pd.notna(average_speed) else None
                    df_train['Elv_numeric'] = pd.to_numeric(df_train['Elv'], errors='coerce')
                    average_elevation = df_train['Elv_numeric'].median()
                    df_main.loc[index, 'Average Elevation (m)'] = round(average_elevation, 2) if pd.notna(average_elevation) else None

                except FileNotFoundError:
                    print(f"  Warning: Train-specific CSV not found for train {train_number} at {train_csv_path}. Skipping.")
                    if 'Average Speed (km/h)' in df_main.columns:
                         df_main.loc[index, 'Average Speed (km/h)'] = None
                    if 'Average Elevation (m)' in df_main.columns:
                         df_main.loc[index, 'Average Elevation (m)'] = None

                except Exception as e:
                    print(f"  Error processing train-specific CSV {train_csv_filename} for train {train_number}: {e}. Skipping data extraction for train {train_number}.")
                    if 'Average Speed (km/h)' in df_main.columns:
                         df_main.loc[index, 'Average Speed (km/h)'] = None
                    if 'Average Elevation (m)' in df_main.columns:
                         df_main.loc[index, 'Average Elevation (m)'] = None
            print("\nFinished processing. Saving updated main CSV...")
            df_main.to_csv(main_csv_path, index=False, encoding='utf-8')
            print(f"Successfully updated and saved the main CSV file: {main_csv_path}")

except FileNotFoundError:
    print(f"An error occurred because the main CSV file was not found at {main_csv_path}")
except Exception as e:
    print(f"An unexpected error occurred during the population process: {e}")

print("Population script finished.")

In [ ]:
print(f"Starting population of Zone of Origin and Zone of Destination in main CSV: {main_csv_path}")

try:
    df_main = pd.read_csv(main_csv_path)
    for index, row in df_main.iterrows():
        train_number = str(row['Train Number']).strip() 
        train_csv_filename = f"{train_number}_table_data.csv"
        train_csv_path = os.path.join(extracted_tables_folder, train_csv_filename)

        try:
            df_train = pd.read_csv(train_csv_path)

            zone_origin = df_train.iloc[0]['Zone']
            df_main.loc[index, 'Zone of Origin'] = str(zone_origin).strip() if pd.notna(zone_origin) else None
            zone_destination = df_train.iloc[-2]['Zone']
            df_main.loc[index, 'Zone of Destination'] = str(zone_destination).strip() if pd.notna(zone_destination) else None


        except Exception as e:
            print(f"  Error processing train {train_number} (CSV: {train_csv_filename}): {e}. Skipping data extraction for this train.")


    print("\nFinished processing. Saving updated main CSV...")
    df_main.to_csv(main_csv_path, index=False, encoding='utf-8')
    print(f"Successfully updated and saved the main CSV file: {main_csv_path}")

except Exception as e:
    print(f"An unexpected error occurred during the population process: {e}")

print("Population script finished.")

COMBINE WITH TRAIN CAPACITY AND OCCUPANCY NUMBERS

In [ ]:
excel_file_path = "Train Data Merged Full Raw 2022 - 2024 - Main.xlsx"

capacity_sheet_identifier = 0 # First sheet (0-indexed)
passengers_sheet_identifier = 2 # Third sheet (0-indexed)

# Define the column renaming mappings.
capacity_rename_map = {
    '1A': 'CAP1A',
    '2A': 'CAP2A',
    '2S': 'CAP2S',
    '3A': 'CAP3A',
    '3E': 'CAP3E',
    'CC': 'CAPCC',
    'EA': 'CAPEA',    
    'EC': 'CAPEC',
    'EV': 'CAPEV',
    'FC': 'CAPFC',
    'SL': 'CAPSL',
    'Grand Capacity': 'CAPTotal' 
}

passengers_rename_map = {
    '1A': 'OCC1A',
    '2A': 'OCC2A',
    '2S': 'OCC2S',
    '3A': 'OCC3A',
    '3E': 'OCC3E',
    'CC': 'OCCCC',
    'EA': 'OCCEA',
    'EC': 'OCCEC',
    'EV': 'OCCEV',
    'FC': 'OCCFC',
    'SL': 'OCCSL',
    'TOTAL': 'OCCTotal' 
}

print(f"Attempting to read Excel file and rename columns: {excel_file_path}")

try:
    df_capacity = pd.read_excel(excel_file_path, sheet_name=capacity_sheet_identifier)
    print(f"Successfully read sheet: {capacity_sheet_identifier} ('Capacity').")

    df_capacity = df_capacity.rename(columns=capacity_rename_map, errors='ignore')
    print("Renamed columns in Capacity DataFrame.")

    df_passengers = pd.read_excel(excel_file_path, sheet_name=passengers_sheet_identifier)
    print(f"Successfully read sheet: {passengers_sheet_identifier} ('Passengers').")

    df_passengers = df_passengers.rename(columns=passengers_rename_map, errors='ignore')
    print("Renamed columns in Passengers DataFrame.")

    print("\nFinished reading Excel file and renaming columns.")

except FileNotFoundError:
    print(f"Error: The Excel file was not found at {excel_file_path}. Please ensure the file name and path are correct.")
except Exception as e:
    print(f"An error occurred while processing the Excel file or renaming columns: {e}")

In [ ]:
main_csv_path = "all_pdfs_extracted_data.csv"

print(f"Starting filtering of Capacity and Passenger data based on train numbers from: {main_csv_path}")

try:
    df_main_train_numbers = pd.read_csv(main_csv_path)

    train_numbers_from_csv_series = df_main_train_numbers['Train Number']

    train_numbers_from_csv = train_numbers_from_csv_series.dropna().astype(str).str.strip().unique()
    print(f"Found {len(train_numbers_from_csv)} unique train numbers in {main_csv_path}.")

    train_no_cap_series = df_capacity['TRAIN NO']
    train_no_pass_series = df_passengers['TRAIN NO']

    df_capacity['TRAIN NO_str'] = train_no_cap_series.dropna().astype(str).str.strip()
    df_capacity_filtered = df_capacity[df_capacity['TRAIN NO_str'].isin(train_numbers_from_csv)].copy()
    df_capacity_filtered = df_capacity_filtered.drop(columns=['TRAIN NO_str'])
    print(f"Filtered Capacity data. {len(df_capacity_filtered)} rows matching train numbers from CSV.")

    df_passengers['TRAIN NO_str'] = train_no_pass_series.dropna().astype(str).str.strip()
    df_passengers_filtered = df_passengers[df_passengers['TRAIN NO_str'].isin(train_numbers_from_csv)].copy()
    df_passengers_filtered = df_passengers_filtered.drop(columns=['TRAIN NO_str'])
    print(f"Filtered Passengers data. {len(df_passengers_filtered)} rows matching train numbers from CSV.")

    print("\nFiltering complete.")
    print(f"Filtered Capacity DataFrame shape: {df_capacity_filtered.shape}")
    print(f"Filtered Passengers DataFrame shape: {df_passengers_filtered.shape}")


except FileNotFoundError as e:
    print(f"A required file was not found: {e}")
except KeyError as e:
    print(f"A required column was not found: {e}. Please check column names in your Excel and CSV files.")
except Exception as e:
    print(f"An unexpected error occurred during filtering: {e}")
    print("Please ensure the Excel file was read correctly in the previous step and has a 'TRAIN NO' column, and that the main CSV file exists and has a 'Train Number' column.")



In [ ]:
capacity_cols_to_aggregate = [col for col in df_capacity_filtered.columns if col != 'TRAIN NO']
passengers_cols_to_aggregate = [col for col in df_passengers_filtered.columns if col != 'TRAIN NO']

df_capacity_aggregated = df_capacity_filtered.loc[:, capacity_cols_to_aggregate].apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0).copy()
df_capacity_aggregated['TRAIN NO'] = df_capacity_filtered['TRAIN NO']
df_capacity_aggregated = df_capacity_aggregated.groupby('TRAIN NO')[capacity_cols_to_aggregate].median().reset_index()

df_passengers_aggregated = df_passengers_filtered.loc[:, passengers_cols_to_aggregate].apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0).copy()
df_passengers_aggregated['TRAIN NO'] = df_passengers_filtered['TRAIN NO']
df_passengers_aggregated = df_passengers_aggregated.groupby('TRAIN NO')[passengers_cols_to_aggregate].median().reset_index()

print("Aggregation complete.")
print(f"Aggregated Capacity DataFrame shape: {df_capacity_aggregated.shape}")
print(f"Aggregated Passengers DataFrame shape: {df_passengers_aggregated.shape}")

In [ ]:
main_csv_path = "all_pdfs_extracted_data.csv"
df_merged = pd.merge(
    pd.merge(
        pd.read_csv(main_csv_path), 
        df_capacity_aggregated[['TRAIN NO'] + [col for col in df_capacity_aggregated.columns if col.startswith('CAP')]], 
        how='left',
        left_on='Train Number', 
        right_on='TRAIN NO' 
    ).drop(columns=['TRAIN NO'], errors='ignore'), 
    df_passengers_aggregated[['TRAIN NO'] + [col for col in df_passengers_aggregated.columns if col.startswith('OCC')]], 
    how='left',
    left_on='Train Number', 
    right_on='TRAIN NO' 
).drop(columns=['TRAIN NO'], errors='ignore') 

print("Merged DataFrames.")
print(f"Merged DataFrame shape: {df_merged.shape}")
df_merged.to_csv('all_pdfs_extracted_data_with_capacity_and_passengers.csv', index=False)
print("Saved merged DataFrame to 'all_pdfs_extracted_data_with_capacity_and_passengers.csv'.")

In [ ]:
print("Starting data transformations on df_merged DataFrame...")

# --- 1. Train Type Encoding ---
if 'Train Type' in df_merged.columns:
    df_merged['Train Type Encoded'], train_type_uniques = pd.factorize(df_merged['Train Type'])
    print("Encoded 'Train Type'. New column 'Train Type Encoded'.")
    print("Train Type Mapping:", dict(zip(train_type_uniques, range(len(train_type_uniques)))))
else:
    print("Warning: 'Train Type' column not found in df_merged. Skipping encoding.")
    df_merged['Train Type Encoded'] = -1 


# --- 2. Travel Days Calculation ---
def count_travel_days(days_str):
    if pd.isna(days_str) or str(days_str).strip() == '':
        return 0 
    
    days_str_lower = str(days_str).lower() 
    if 'daily' in days_str_lower or 'all days' in days_str_lower:
        return 7
    
    days_list = [day.strip() for day in days_str.split(',') if day.strip()]
    return len(days_list)

if 'Days of Operation' in df_merged.columns:
    df_merged['Travel Days'] = df_merged['Days of Operation'].apply(count_travel_days)
    print("Calculated 'Travel Days'. New column 'Travel Days'.")
else:
    print("Warning: 'Days of Operation' column not found in df_merged. Skipping 'Travel Days' calculation.")
    df_merged['Travel Days'] = 0 

# --- 4. Zone of Origin Encoding ---
if 'Zone of Origin' in df_merged.columns:
    df_merged['Zone of Origin Encoded'], zone_origin_uniques = pd.factorize(df_merged['Zone of Origin'])
    print("Encoded 'Zone of Origin'. New column 'Zone of Origin Encoded'.")
    print("Zone of Origin Mapping:", dict(zip(zone_origin_uniques, range(len(zone_origin_uniques)))))
else:
    print("Warning: 'Zone of Origin' column not found in df_merged. Skipping encoding.")
    df_merged['Zone of Origin Encoded'] = -1 


# --- 5. Zone of Destination Encoding ---
if 'Zone of Destination' in df_merged.columns:
    df_merged['Zone of Destination Encoded'], zone_destination_uniques = pd.factorize(df_merged['Zone of Destination'])
    print("Encoded 'Zone of Destination'. New column 'Zone of Destination Encoded'.")
    print("Zone of Destination Mapping:", dict(zip(zone_destination_uniques, range(len(zone_destination_uniques)))))
else:
    print("Warning: 'Zone of Destination' column not found in df_merged. Skipping encoding.")
    df_merged['Zone of Destination Encoded'] = -1 

print("\nAll transformations complete on df_merged.")
print(df_merged.head(3))

In [ ]:
def categorize_weekend_weekday(days_str):
    if pd.isna(days_str) or str(days_str).strip() == '':
        return None 

    days_str_lower = str(days_str).strip().lower() 

    if 'daily' in days_str_lower or 'all days' in days_str_lower:
        return 20 

    days_list_cleaned_lower = [day.strip().lower() for day in days_str.split(',') if day.strip()]

    has_weekend = any(day in ['sun', 'sat'] for day in days_list_cleaned_lower)

    if has_weekend:
        return 20 

    weekday_abbreviations = ['mon', 'tue', 'wed', 'thu', 'fri']
    has_weekday = any(day in weekday_abbreviations for day in days_list_cleaned_lower)

    if has_weekday:
         return 10 

    return None 

if 'Days of Operation' in df_merged.columns:
    df_merged['Weekend/Day'] = df_merged['Days of Operation'].astype(str).apply(categorize_weekend_weekday)
else:
    df_merged['Weekend/Day'] = None 

In [ ]:
def classify_time(t):
    if pd.isnull(t):
        return None  
    hour = int(t.split(":")[0])
    if 4 <= hour <= 18:
        return 10  # Day
    else:
        return 20  # Night

# Apply to your DataFrame
df_merged['Night/Day-Dep'] = df_merged['Dep Time'].apply(classify_time)
df_merged['Night/Day-Arr'] = df_merged['Arr Time'].apply(classify_time)


In [ ]:
df_merged.to_csv('trainfulldata.csv', index=False)
